In [ ]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import joblib
import math
import os
import tensorflow as tf

app = Flask(__name__)

# --- Configuration ---
HOTEL_CSV = 'final_data.csv'
MODEL_FILE = 'quality_prediction_model.keras'
HOTEL_DB_FILE = 'hotel_data_processed.joblib'

df_hotels = None
model = None
hotel_features = None
global_max_price = 5000
available_amenities_list = []

def clean_currency(x):
    if pd.isna(x): return 0.0
    s = str(x).replace('$', '').replace('€', '').replace(',', '').strip()
    try: return float(s)
    except: return 0.0

def load_system():
    global df_hotels, model, hotel_features, global_max_price, available_amenities_list
    if os.path.exists(HOTEL_CSV):
        try:
            df_hotels = pd.read_csv(HOTEL_CSV)
            df_hotels['Latitude'] = pd.to_numeric(df_hotels['Latitude'], errors='coerce')
            df_hotels['Longitude'] = pd.to_numeric(df_hotels['Longitude'], errors='coerce')
            df_hotels['price'] = df_hotels['price'].apply(clean_currency)
            global_max_price = df_hotels['price'].max()
            df_hotels['rating'] = pd.to_numeric(df_hotels['rating'], errors='coerce').fillna(0)
            
            # Reset index to ensure we have a clean ID
            df_hotels.reset_index(inplace=True, drop=True) 

            exclude_cols = ['name', 'location', 'price', 'rating', 'Latitude', 'Longitude', 'images', 'image1', 'image2', 'image3', 'image4', 'image5', 'gps_link', 'id', 'Unnamed: 0', 'amenities', 'Unnamed: 12']
            potential_cols = [col for col in df_hotels.columns if col not in exclude_cols]
            for col in potential_cols:
                if pd.api.types.is_numeric_dtype(df_hotels[col]):
                    if len(df_hotels[col].unique()) <= 3: available_amenities_list.append(col)
            
            print(f"✅ Data Loaded. Max Price: {global_max_price}")
        except Exception as e:
             print(f"❌ Error loading CSV: {e}")
             df_hotels = pd.DataFrame()

    if os.path.exists(MODEL_FILE) and os.path.exists(HOTEL_DB_FILE):
        try:
            model = tf.keras.models.load_model(MODEL_FILE)
            hotel_db = joblib.load(HOTEL_DB_FILE)
            hotel_features = hotel_db['features']
            preds = model.predict(hotel_features, verbose=0).flatten()
            df_hotels['ai_score'] = preds
        except Exception as e:
            print(f"⚠️ AI Model Error: {e}")
            df_hotels['ai_score'] = 4.0 
    else:
        if df_hotels is not None: df_hotels['ai_score'] = 4.0

with app.app_context():
    load_system()

def haversine(lat1, lon1, lat2, lon2):
    if any(pd.isna(x) for x in [lat1, lon1, lat2, lon2]): return 99999
    R = 6371
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon/2) * math.sin(dLon/2)
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

# --- Helper to Format Data ---
def format_hotel_response(row, distance=0):
    # Collect images
    all_images = []
    if isinstance(row.get('images'), str):
        urls = str(row['images']).split(',')
        for u in urls:
            if len(u.strip()) > 10: all_images.append(u.strip())

    for i in range(1, 6):
        col = f"image{i}"
        val = row.get(col)
        if isinstance(val, str) and len(val) > 10:
            if val not in all_images: all_images.append(val.strip())

    main_image = all_images[0] if all_images else "https://placehold.co/600x400?text=No+Image"

    return {
        'id': int(row.name), # Important: Send the index as ID
        'name': str(row['name']),
        'price': float(row['price']),
        'rating': float(row['rating']),
        'lat': float(row['Latitude']),
        'lng': float(row['Longitude']),
        'image_url': main_image,
        'all_images': all_images,
        'distance_km': round(distance, 1),
        'match_percent': int((row.get('ai_score', 4.0) / 5.0) * 100),
        'amenities': str(row.get('amenities', ''))[:300],
        'location': str(row.get('location', ''))
    }

# --- Routes ---
@app.route('/')
def index(): return render_template('index.html')

@app.route('/search_panel')
@app.route('/search_panel.html')
def search_panel_route(): return render_template('search_panel.html')

# ✅ صفحة التفاصيل الجديدة
@app.route('/hotel/<int:hotel_id>')
def hotel_details(hotel_id):
    if df_hotels is not None and 0 <= hotel_id < len(df_hotels):
        row = df_hotels.iloc[hotel_id]
        hotel_data = format_hotel_response(row)
        return render_template('details.html', hotel=hotel_data)
    else:
        return "Hotel not found", 404

@app.route('/api/metadata')
def get_metadata():
    return jsonify({'max_price': global_max_price, 'amenities': available_amenities_list})

@app.route('/api/hotels')
def get_all_hotels():
    if df_hotels is None or df_hotels.empty: return jsonify([])
    valid = df_hotels.dropna(subset=['Latitude', 'Longitude'])
    response_data = [format_hotel_response(row) for _, row in valid.head(500).iterrows()]
    return jsonify(response_data)

@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        data = request.get_json()
        user_lat = data.get('lat')
        user_lng = data.get('lng')
        radius = float(data.get('radius', 50))
        amenities = data.get('amenities', [])

        if user_lat is None: return jsonify({'status': 'error', 'message': 'Location missing'})
        
        results = df_hotels.dropna(subset=['Latitude', 'Longitude']).copy()
        results['distance_km'] = results.apply(
            lambda row: haversine(user_lat, user_lng, row['Latitude'], row['Longitude']), axis=1
        )

        nearby = results[results['distance_km'] <= radius].copy()
        if amenities:
            for feature in amenities:
                if feature in nearby.columns:
                    nearby = nearby[nearby[feature] == 1]

        if nearby.empty:
             if len(amenities) > 0: 
                 return jsonify({'status': 'success', 'data': [], 'message': 'No hotels match all these amenities.'})
             nearby = results.sort_values('distance_km').head(10).copy()

        max_dist = nearby['distance_km'].max() + 0.1
        nearby['dist_score'] = 1 - (nearby['distance_km'] / max_dist)
        nearby['quality_norm'] = nearby['ai_score'] / 5.0
        nearby['final_rank'] = (nearby['quality_norm'] * 0.7) + (nearby['dist_score'] * 0.3)

        top_hotels = nearby.sort_values('final_rank', ascending=False).head(20)
        response_data = [format_hotel_response(row, row['distance_km']) for _, row in top_hotels.iterrows()]

        return jsonify({'status': 'success', 'data': response_data})

    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)